<a href="https://colab.research.google.com/github/FarshadAmiri/Structural_Engineering/blob/master/CrossVal1_mahdi_structures_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import KFold
import pandas as pd
from sklearn import metrics
from scipy.stats import zscore
import numpy as np
import time

In [ ]:
# Dataset = pd.read_excel(r'https://github.com/FarshadAmiri/Structrual_Design_Optimization/blob/master/Dataset%20(213%20samples).xlsx?raw=true')
Dataset = pd.read_excel(r'https://github.com/FarshadAmiri/Structural_Engineering/blob/master/D600.xlsx?raw=true')

# Dataset = pd.read_excel(r'https://github.com/FarshadAmiri/Structural_Engineering/blob/master/D608-m.xlsx?raw=true')
cols = ['Mu33','Mu22','Pu','L','Cb','Delta']

for col in cols:
    col_zscore = col + '_zscore'
    Dataset[col_zscore] = (Dataset[col] - Dataset[col].mean())/Dataset[col].std(ddof=0)


In [ ]:
#Defining Trainig data and Test Data
shuffled_dataset = Dataset.reindex(np.random.permutation(Dataset.index))

train_df = shuffled_dataset[:605]
test_df  = shuffled_dataset[530:]

In [ ]:
# Assigning features and targets (z-score)
x = np.column_stack((train_df.Mu33_zscore.values , train_df.Mu22_zscore.values, train_df.Pu_zscore.values, train_df.L_zscore.values, train_df.Cb_zscore.values, train_df.Delta_zscore.values, train_df.L1tb.values))
y = np.column_stack((train_df.h.values, train_df.b.values, train_df.tf.values, train_df.tw.values))

test_x = np.column_stack((test_df.Mu33_zscore.values , test_df.Mu22_zscore.values, test_df.Pu_zscore.values, test_df.L_zscore.values, test_df.Cb_zscore.values, test_df.Delta_zscore.values,  test_df.L1tb.values))
test_y = np.column_stack((test_df.h.values, test_df.b.values, test_df.tf.values, test_df.tw.values))

In [ ]:
# K-fold cross validation
kf = KFold(5, shuffle=True, random_state=42)

In [ ]:
oos_y = []
oos_pred = []

In [ ]:
fold = 0
for train,val in kf.split(x):
    fold+=1
    print(f"Fold #{fold}")
        
    x_train = x[train]
    y_train = y[train]
    x_test = x[val]
    y_test = y[val]


    Lambda = 0.01
    nn = 128
    model = keras.Sequential([
    keras.layers.Dense(nn,input_shape=(7,),activation='relu',kernel_regularizer= tf.keras.regularizers.L2(l2=Lambda)),
    keras.layers.Dense(nn,activation='relu', kernel_regularizer= tf.keras.regularizers.L2(l2=Lambda)),
    keras.layers.Dense(nn,activation='relu', kernel_regularizer= tf.keras.regularizers.L2(l2=Lambda)),
    keras.layers.Dense(nn,activation='relu', kernel_regularizer= tf.keras.regularizers.L2(l2=Lambda)),
    keras.layers.Dense(nn,activation='relu', kernel_regularizer= tf.keras.regularizers.L2(l2=Lambda)),
    keras.layers.Dense(nn,activation='relu', kernel_regularizer= tf.keras.regularizers.L2(l2=Lambda)),
    keras.layers.Dense(nn,activation='relu', kernel_regularizer= tf.keras.regularizers.L2(l2=Lambda)),
    keras.layers.Dense(nn,activation='relu', kernel_regularizer= tf.keras.regularizers.L2(l2=Lambda)),
    keras.layers.Dense(nn,activation='relu', kernel_regularizer= tf.keras.regularizers.L2(l2=Lambda)),
    keras.layers.Dense(4, activation='linear', kernel_regularizer= tf.keras.regularizers.L2(l2=Lambda))])

    model.compile(
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.002),
              loss = 'mse',
              metrics = tf.keras.metrics.RootMeanSquaredError(name='rmse', dtype=None)
              # metrics=['accuracy']
              )
    
    model.fit(x_train,y_train, validation_data=(x_test,y_test), verbose=0,
              epochs=300)
    
    pred = model.predict(x_test)
    
    oos_y.append(y_test)
    oos_pred.append(pred)    

    # Measure this fold's RMSE
    score = np.sqrt(metrics.mean_squared_error(pred,y_test))
    print(f"Fold score (RMSE): {score}")

Fold #1
Fold score (RMSE): 5.217110123803416
Fold #2
Fold score (RMSE): 3.494557750176774
Fold #3
Fold score (RMSE): 4.9865179648227125
Fold #4
Fold score (RMSE): 5.552159674631867
Fold #5
Fold score (RMSE): 6.588827618888964


In [ ]:
# Build the oos prediction list and calculate the error.
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
score = np.sqrt(metrics.mean_squared_error(oos_pred,oos_y))
print(f"Final, out of sample score (RMSE): {score}")

Final, out of sample score (RMSE): 5.263737451103628


In [ ]:
df = df = pd.DataFrame(data=None)
oos_y = pd.DataFrame(oos_y,columns=['h','b','tf','tw'])
oos_pred = pd.DataFrame(oos_pred,columns=['h_pred','b_pred','tf_pred','tw_pred'])
oosDF = pd.concat( [df, oos_y, oos_pred],axis=1 )
#oosDF.to_csv(filename_write,index=False)

In [ ]:
# OOS stands for Out of Sample data which is our validation data of each fold
oosDF

,h,b,tf,tw,h_pred,b_pred,tf_pred,tw_pred
0,31.1150,30.5816,1.7018,1.0922,27.209240,24.739487,2.188481,1.482775
1,15.2146,15.2146,0.6604,0.5842,15.570258,12.955834,1.081362,0.817800
2,56.9468,45.4406,12.4714,7.7978,57.176792,46.578362,11.946263,7.384623
3,97.1804,43.0911,6.1976,3.4544,99.394142,44.001816,7.335848,4.145467
4,86.8172,40.2844,3.5560,2.1082,99.066566,43.769409,3.262741,1.832905
...,...,...,...,...,...,...,...,...
595,97.7138,42.9514,8.9916,5.0038,95.356163,44.822880,8.616078,5.220346
596,60.2996,17.8816,1.4986,1.0922,56.170811,19.319082,1.583043,1.276423
597,99.9998,45.0850,3.5941,1.9050,91.925201,43.488888,4.072060,2.589909
598,107.8230,46.0502,11.5062,6.4008,100.282204,47.409351,11.223021,6.678576
